# Adjusting, Renaming and Restructuring

In [2]:
import pymrio

In [3]:
mrio = pymrio.load_test()

We can use several functions to get a quick overview over the MRIO system:

In [4]:
mrio.get_sectors()

Index(['food', 'mining', 'manufactoring', 'electricity', 'construction',
       'trade', 'transport', 'other'],
      dtype='object', name='sector')

In [5]:
mrio.get_regions()

Index(['reg1', 'reg2', 'reg3', 'reg4', 'reg5', 'reg6'], dtype='object', name='region')

In [6]:
mrio.get_Y_categories()

Index(['Final consumption expenditure by households',
       'Final consumption expenditure by non-profit organisations serving households (NPISH)',
       'Final consumption expenditure by government',
       'Gross fixed capital formation', 'Changes in inventories',
       'Changes in valuables', 'Export'],
      dtype='object', name='category')

A list of available satellite accounts can be obtained by

In [7]:
mrio.get_extensions()

<generator object IOSystem.get_extensions at 0x7f36f120c4a0>

this returns a generator over all extensions, to just get the names which can be used to loop over the extensions. To just get the names:

In [8]:
list(mrio.get_extensions())

['factor_inputs', 'emissions']

All the names returned are literally just names and can easily be renamed.
To rename sectors or regions just pass a dictionary with the new names:

In [9]:
mrio.rename_regions({"reg1": "REGION A", "reg2": "REGION B"})

In [10]:
mrio.get_regions()

Index(['REGION A', 'REGION B', 'reg3', 'reg4', 'reg5', 'reg6'], dtype='object', name='region')

Renaming sectors or Y categories works the same way

In [11]:
mrio.rename_sectors({"mining": "dwarf business"})
mrio.rename_Y_categories({"Final consumption expenditure by households": "fin_house"})

In [12]:
mrio.get_sectors()
mrio.get_Y_categories()

Index(['fin_house',
       'Final consumption expenditure by non-profit organisations serving households (NPISH)',
       'Final consumption expenditure by government',
       'Gross fixed capital formation', 'Changes in inventories',
       'Changes in valuables', 'Export'],
      dtype='object', name='category')

Some MRIOs come with a selections of names which can be used for renaming.
These can be obtained via 'get_classification', passing the name of the IO (pass None to get a list of available classifications).

In [13]:
mrio_class = pymrio.get_classification(mrio_name="test")

The classification data contains different names and aggregation levels for the sectors and final demand categories. The easiest way to explore the classification data is by using the autocomplete functionality. Depending on the editor your are using this might work with typing mrio_class.sectors. pressing Tab or Ctrl-Space. The same works for mrio_class.finaldemand.

Alternatively it is possible to just print the underlying dataframes with (similar for finaldemand)

In [14]:
mrio_class.sectors

,TestMrioNumber,TestMrioName,TestMrioCode,Type
0,1,food,sec1,eat
1,2,mining,sec2,dig
2,3,manufactoring,sec3,build
3,4,electricity,sec4,util
4,5,construction,sec5,build
5,6,trade,sec6,marg
6,7,transport,sec7,marg
7,8,other,sec8,misc


This can be used to generate dictionaries for renaming the sectors of the MRIO, eg with:

In [19]:
conv_dict = mrio_class.get_sector_dict(mrio_class.sectors.TestMrioName, mrio_class.sectors.TestMrioCode)
conv_dict

{'food': 'sec1',
 'mining': 'sec2',
 'manufactoring': 'sec3',
 'electricity': 'sec4',
 'construction': 'sec5',
 'trade': 'sec6',
 'transport': 'sec7',
 'other': 'sec8'}

This can then be used for renaming the sectors:

In [16]:
mrio = pymrio.load_test()

In [21]:
mrio.rename_sectors(conv_dict)
mrio.get_sectors()

Index(['sec1', 'sec2', 'sec3', 'sec4', 'sec5', 'sec6', 'sec7', 'sec8'], dtype='object', name='sector')

In the mrio_class.sectors you will also find an entry 'Type' which represents a many to one mapping. This can be used for aggregating the MRIO and is explained [in the aggregation tutorial](aggregation_examples.ipynb#Aggregation-by-renaming).